In [1]:
import pandas as pd
import fastText as ft
import os
import numpy as np
import sklearn

In [19]:
df = pd.read_csv("amazon_embeddings_ver2.csv",index_col = None)

In [20]:
df.drop(columns = 'Unnamed: 0',inplace=True)

### Load FastText model and add sentence embeddings to Dataframe

In [9]:
model = ft.load_model('amazon.ftz')

In [21]:
embed = []
for i in df['product_name']:
    embed.append(model.get_sentence_vector((str(i))))
df['sent_embed'] = embed

In [22]:
c ={'active': 1,
 'athletic shoes': 2,
 'boots': 3,
 'coats, jackets and vests': 4,
 'dresses': 5,
 'fashion hoodies and sweatshirts': 6,
 'fashion sneakers': 7,
 'flats': 8,
 'jeans': 9,
 'jumpsuits, rompers and overalls': 10,
 'leggings': 11,
 'lingerie and sleepwear': 12,
 'mules and clogs': 13,
 'outdoor shoes': 14,
 'oxford shoes': 15,
 'pants': 16,
 'pumps': 17,
 'sandals': 18,
 'shorts': 19,
 'skirts': 20,
 'slippers': 21,
 'socks and hosiery': 22,
 'suiting and blazers': 23,
 'sweaters': 24,
 'swimsuits and cover ups': 25,
 'tops and tees': 26}

### Add Column "Label" as integer

In [23]:
df['label']=df['category'].apply(lambda x: int(c[x]))

In [25]:
df.head()

,brand,category,price,product_name,sent_embed,label
0,Domy,swimsuits and cover ups,22.99,plus size swimwear retro halter top vintage on...,"[-201.85323, -127.87654, 0.8215461, 323.24844,...",25
1,LE CHÂTEAU,pumps,49.97,women's classic leather pointy toe slingback pump,"[7.2089806, 124.064354, -22.363012, 173.3266, ...",17
2,DongDong,dresses,7.82,dress ladies short casual pocket summer sleeve...,"[39.89494, 102.13796, -127.38166, -96.415955, ...",5
3,NaN,jeans,105.00,super stretch skinny fit distressed jeans wfla...,"[40.05252, 124.04834, -93.65457, -474.3166, -1...",9
4,NaN,pumps,42.99,women's total motion salima dress pump,"[41.03999, 247.08846, 340.8324, -8.21388, -328...",17


### Separate Data into X and y

In [46]:
X = np.array(df.sent_embed.tolist())
y = df['label']
y = y.values.reshape(-1,1)
X.shape

(257613, 100)

In [15]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.3,random_state=100)

### Logistic Regression

In [29]:
from sklearn.linear_model import LogisticRegression
logmodel = LogisticRegression()
logmodel.fit(X_train,y_train)
predictions = logmodel.predict(X_test)

In [34]:
from sklearn.metrics import classification_report,confusion_matrix
print(classification_report(y_test,predictions))

             precision    recall  f1-score   support

          1       0.83      0.82      0.82      3088
          2       0.82      0.77      0.79      3000
          3       0.90      0.88      0.89      2939
          4       0.96      0.95      0.96      3052
          5       0.94      0.97      0.96      2920
          6       0.96      0.96      0.96      2928
          7       0.92      0.90      0.91      3038
          8       0.95      0.95      0.95      2935
          9       0.99      0.98      0.98      3023
         10       0.97      0.98      0.98      2918
         11       0.97      0.97      0.97      3046
         12       0.96      0.93      0.94      3129
         13       0.96      0.95      0.96      2755
         14       0.76      0.83      0.79      3051
         15       0.96      0.94      0.95      2540
         16       0.93      0.95      0.94      3010
         17       0.94      0.96      0.95      3040
         18       0.89      0.91      0.90   

### Cross-validation

In [47]:
from sklearn.model_selection import KFold
kfLog = KFold(n_splits=5,random_state = 100, shuffle=False) 
for train_index, test_index in kfLog.split(X):
    print("Train:", train_index, "Test:", test_index)
    X_train,X_test = X[train_index],X[test_index]
    y_train,y_test = y[train_index],y[test_index]
    logmodel.fit(X_train,y_train)
    predictions = logmodel.predict(X_test)
    print(classification_report(y_test,predictions))

Train: [ 51523  51524  51525 ... 257610 257611 257612] Test: [    0     1     2 ... 51520 51521 51522]
Train: [     0      1      2 ... 257610 257611 257612] Test: [ 51523  51524  51525 ... 103043 103044 103045]
Train: [     0      1      2 ... 257610 257611 257612] Test: [103046 103047 103048 ... 154566 154567 154568]
Train: [     0      1      2 ... 257610 257611 257612] Test: [154569 154570 154571 ... 206088 206089 206090]
Train: [     0      1      2 ... 206088 206089 206090] Test: [206091 206092 206093 ... 257610 257611 257612]


In [50]:
score_array = []
avg_score = np.mean(score_array)
print(avg_score)

c:\Anaconda3\lib\site-packages\numpy\core\fromnumeric.py:2957: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)


nan


c:\Anaconda3\lib\site-packages\numpy\core\_methods.py:80: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [52]:
kf = KFold(n_splits=5)
from sklearn.metrics import precision_recall_fscore_support
KFold(n_splits=5, random_state=100, shuffle=False)

clf_LogReg=LogisticRegression()

score_array =[]
for train_index, test_index in kf.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    clf=clf_LogReg.fit(X_train,y_train)
    y_pred = clf.predict(X_test)
    score_array.append(precision_recall_fscore_support(y_test, y_pred, average=None))

avg_score = np.mean(score_array,axis=0)
print(avg_score)


c:\Anaconda3\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[[8.33177236e-01 8.30563343e-01 9.11154606e-01 9.60257511e-01
  9.48235431e-01 9.63482940e-01 9.12351132e-01 9.44544426e-01
  9.85914120e-01 9.79024299e-01 9.63882877e-01 9.56601567e-01
  9.58596948e-01 7.54573891e-01 9.62394451e-01 9.35518103e-01
  9.51109947e-01 8.94221139e-01 9.84646252e-01 9.67166809e-01
  9.69361149e-01 9.36666567e-01 9.36572448e-01 9.59829337e-01
  9.90070101e-01 9.31437167e-01]
 [8.16907684e-01 7.76333884e-01 8.72213740e-01 9.48577143e-01
  9.69241263e-01 9.60060024e-01 9.04145160e-01 9.55012807e-01
  9.81798421e-01 9.83457389e-01 9.70836739e-01 9.37624794e-01
  9.53610235e-01 8.36721417e-01 9.35160017e-01 9.39660500e-01
  9.58243219e-01 9.07240746e-01 9.80416900e-01 9.81233200e-01
  9.61853124e-01 9.51482651e-01 9.09646903e-01 9.78767481e-01
  9.79770349e-01 9.55545330e-01]
 [8.24930532e-01 8.02361842e-01 8.91212165e-01 9.54371362e-01
  9.58609537e-01 9.61745846e-01 9.08223885e-01 9.49723226e-01
  9.83849668e-01 9.81230469e-01 9.67335488e-01 9.47008732e-01
  9.

In [57]:
avg_score[3][0]

2000.0